# Section 1: Password Strength Analysis
Function that scores password strength by checking length thresholds, character variety, common password usage, and approximate entropy.

In [19]:
from __future__ import annotations

import math
import string
from dataclasses import dataclass
from typing import Dict

COMMON_PASSWORDS = {
    "password",
    "123456",
    "12345678",
    "qwerty",
    "abc123",
    "letmein",
    "iloveyou",
    "admin",
    "welcome",
    "monkey",
}

@dataclass(frozen=True)
class PasswordReport:
    password: str
    length: int
    length_score: int
    character_sets: Dict[str, bool]
    variety_score: int
    entropy_bits: float
    is_common: bool
    total_score: int
    verdict: str

def analyze_password(password: str) -> PasswordReport:
    """Return a simple strength report for the supplied password."""
    length = len(password)
    length_score = int(length >= 8) + int(length >= 12)

    character_sets = {
        "lowercase": any(ch in string.ascii_lowercase for ch in password),
        "uppercase": any(ch in string.ascii_uppercase for ch in password),
        "digits": any(ch in string.digits for ch in password),
        "symbols": any(ch in string.punctuation for ch in password),
    }

    variety_score = sum(character_sets.values())

    pool_size = 0
    if character_sets["lowercase"]:
        pool_size += len(string.ascii_lowercase)
    if character_sets["uppercase"]:
        pool_size += len(string.ascii_uppercase)
    if character_sets["digits"]:
        pool_size += len(string.digits)
    if character_sets["symbols"]:
        pool_size += len(string.punctuation)

    entropy_bits = 0.0
    if pool_size > 0 and length > 0:
        entropy_bits = length * math.log2(pool_size)

    is_common = password.lower() in COMMON_PASSWORDS

    total_score = length_score + variety_score
    if is_common:
        total_score = max(0, total_score - 2)

    if total_score >= 6 and entropy_bits >= 60:
        verdict = "strong"
    elif total_score >= 4 and entropy_bits >= 40:
        verdict = "moderate"
    else:
        verdict = "weak"

    return PasswordReport(
        password=password,
        length=length,
        length_score=length_score,
        character_sets=character_sets,
        variety_score=variety_score,
        entropy_bits=round(entropy_bits, 2),
        is_common=is_common,
        total_score=total_score,
        verdict=verdict,
    )

In [20]:
sample_passwords = [
    "password",
    "Tru5t!nN0ne",
    "short",
    "N3tw0rk$ecurity2025!",
    "Pass123",
    "MyP@ssw0rd!"
]

for pwd in sample_passwords:
    report = analyze_password(pwd)
    print(f"Password: {pwd}")
    print(f"  Length score: {report.length_score}")
    print(f"  Variety score: {report.variety_score}")
    print(f"  Entropy (bits): {report.entropy_bits}")
    print(f"  Common password: {report.is_common}")
    print(f"  Total score: {report.total_score}")
    print(f"  Verdict: {report.verdict}\n")

Password: password
  Length score: 1
  Variety score: 1
  Entropy (bits): 37.6
  Common password: True
  Total score: 0
  Verdict: weak

Password: Tru5t!nN0ne
  Length score: 1
  Variety score: 4
  Entropy (bits): 72.1
  Common password: False
  Total score: 5
  Verdict: moderate

Password: short
  Length score: 0
  Variety score: 1
  Entropy (bits): 23.5
  Common password: False
  Total score: 1
  Verdict: weak

Password: N3tw0rk$ecurity2025!
  Length score: 2
  Variety score: 4
  Entropy (bits): 131.09
  Common password: False
  Total score: 6
  Verdict: strong

Password: Pass123
  Length score: 0
  Variety score: 3
  Entropy (bits): 41.68
  Common password: False
  Total score: 3
  Verdict: weak

Password: MyP@ssw0rd!
  Length score: 1
  Variety score: 4
  Entropy (bits): 72.1
  Common password: False
  Total score: 5
  Verdict: moderate



## Why "MyP@ssw0rd!" Scores Higher Than "Pass123"
The second password is longer and uses all four character sets (uppercase, lowercase, digits, symbols), which boosts both the variety score and the estimated entropy. "Pass123" lacks a symbol, hits only seven characters, and therefore earns fewer points for length and variety, leading to a weaker overall verdict.

## Section 2: Don't Store Passwords, Store Hashes!

In [21]:
import hashlib
import time
from typing import Tuple

import bcrypt

def hash_md5(password: str) -> str:
    """Return the MD5 hex digest of the supplied password."""
    return hashlib.md5(password.encode("utf-8")).hexdigest()

def hash_sha256(password: str) -> str:
    """Return the SHA-256 hex digest of the supplied password."""
    return hashlib.sha256(password.encode("utf-8")).hexdigest()

def hash_bcrypt(password: str, rounds: int = 12) -> bytes:
    """Generate a bcrypt hash using a random salt and configurable cost."""
    salt = bcrypt.gensalt(rounds)
    return bcrypt.hashpw(password.encode("utf-8"), salt)

def verify_bcrypt(password: str, hashed: bytes) -> bool:
    """Check a password against a stored bcrypt hash."""
    return bcrypt.checkpw(password.encode("utf-8"), hashed)

def timed_hash(func, password: str) -> Tuple[float, str]:
    """Utility to time how long a hashing function takes and return its output."""
    start = time.perf_counter()
    result = func(password)
    elapsed = time.perf_counter() - start
    return elapsed, result

In [22]:
test_password = "MyP@ssw0rd!"

md5_time, md5_hash = timed_hash(hash_md5, test_password)
sha_time, sha_hash = timed_hash(hash_sha256, test_password)
bcrypt_time, bcrypt_hash = timed_hash(lambda pwd: hash_bcrypt(pwd, rounds=12), test_password)

print("MD5")
print(f"  Hash: {md5_hash}")
print(f"  Time (s): {md5_time:.6f}\n")

print("SHA-256")
print(f"  Hash: {sha_hash}")
print(f"  Time (s): {sha_time:.6f}\n")

print("bcrypt")
print(f"  Hash: {bcrypt_hash.decode('utf-8')}")
print(f"  Time (s): {bcrypt_time:.6f}")
print(f"  Verifies original password: {verify_bcrypt(test_password, bcrypt_hash)}")
print(f"  Verifies wrong password: {verify_bcrypt('wrongpass', bcrypt_hash)}")

MD5
  Hash: 42c853d6bbd0cfddc2d0978df437fa97
  Time (s): 0.000028

SHA-256
  Hash: e493c394a28652900d73f0fc7e6713840b1af0ab1f3fd9c5878d82e5f753c6c1
  Time (s): 0.000008

bcrypt
  Hash: $2b$12$77hsTTGFTgn9P52d/H1za.o.wKkQrPEm3RE/C3zxXMSZj7LX5Oah6
  Time (s): 0.218320
  Verifies original password: True
  Verifies wrong password: False
  Verifies original password: True
  Verifies wrong password: False


### Interpreting the Hash Outputs
MD5 and SHA-256 return short fixed-length hexadecimal strings, while the bcrypt result has the form `$2b$12$...`. That prefix encodes the algorithm version (`2b`), the work factor (`12` rounds here), followed by the salt and the final hash. Using a fast algorithm like MD5 or SHA-256 for passwords is dangerous because attackers can try billions of guesses per second once they have the hashes, making brute-force attacks far more feasible. Bcrypt’s deliberate slowness and embedded salt dramatically raise the cost of each guess.

## Section 3: Adding Salt and Pepper
Salts stop pre-computed rainbow-table attacks by making each hash unique, while a secret pepper adds an extra layer attackers cannot recover from the database alone.

In [23]:
import os
import secrets

def simple_hash(password: str) -> str:
    """Fast hash used to demonstrate salt/pepper concepts (uses SHA-256)."""
    return hashlib.sha256(password.encode("utf-8")).hexdigest()

def hash_with_salt(password: str, salt: bytes) -> str:
    return hashlib.sha256(salt + password.encode("utf-8")).hexdigest()

def hash_with_salt_and_pepper(password: str, salt: bytes, pepper: str) -> str:
    return hashlib.sha256(pepper.encode("utf-8") + salt + password.encode("utf-8")).hexdigest()

def demo_salt_and_pepper():
    password = "user123password"
    pepper = os.environ.get("APP_PEPPER", "SuperSecretPepperValue")

    print("Without salt (two hashes of the same password):")
    hash_a = simple_hash(password)
    hash_b = simple_hash(password)
    print(f"  Hash 1: {hash_a}")
    print(f"  Hash 2: {hash_b}")
    print("  Matches: ", hash_a == hash_b)
    print()

    print("With random salt:")
    salt1 = secrets.token_bytes(16)
    salt2 = secrets.token_bytes(16)
    salted_hash_1 = hash_with_salt(password, salt1)
    salted_hash_2 = hash_with_salt(password, salt2)
    print(f"  Salt 1: {salt1.hex()}")
    print(f"  Salted Hash 1: {salted_hash_1}")
    print(f"  Salt 2: {salt2.hex()}")
    print(f"  Salted Hash 2: {salted_hash_2}")
    print("  Matches: ", salted_hash_1 == salted_hash_2)
    print()

    print("With salt + pepper:")
    peppered_hash_1 = hash_with_salt_and_pepper(password, salt1, pepper)
    peppered_hash_2 = hash_with_salt_and_pepper(password, salt2, pepper)
    print(f"  Pepper (hidden in config): {pepper}")
    print(f"  Salted+Peppered Hash 1: {peppered_hash_1}")
    print(f"  Salted+Peppered Hash 2: {peppered_hash_2}")
    print("  Matches with same salt: ", peppered_hash_1 == hash_with_salt_and_pepper(password, salt1, pepper))

In [24]:
demo_salt_and_pepper()

Without salt (two hashes of the same password):
  Hash 1: 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
  Hash 2: 6384e02d9a47ac61c64950915c878ea54cc91a3d9c11d7497818017db8356d2a
  Matches:  True

With random salt:
  Salt 1: 768194e21d07d66d831634e233dd4d82
  Salted Hash 1: 150df56ae827eaeb522747af197fcdfca5036c37264107bcdcc3e9ac254a51ec
  Salt 2: d849c5464acaa2645322c404204597c6
  Salted Hash 2: 1fdb46b51b595f3796aef6768bede47e31365fbac83c0a41d75a1f35fef18784
  Matches:  False

With salt + pepper:
  Pepper (hidden in config): SuperSecretPepperValue
  Salted+Peppered Hash 1: 8955afba9dabccf12edd01868417a6db977ea47b7c00d3e3020e8131396c7da8
  Salted+Peppered Hash 2: 3222e6b6b3f0394a443f44324f12daf53d13b6f50b8703fcf5c0291c8d68d5b8
  Matches with same salt:  True


### Salt + Pepper Takeaways
Re-hashing the same password without a salt always reproduces the same digest, which rainbow tables exploit. Random per-user salts ensure identical passwords hash differently, and a hidden pepper means even if attackers grab the database, they still cannot verify guesses without the extra secret stored off-database.

### If the Database Is Stolen
- Without salt (just SHA-256): the attacker can look up each hash in pre-built rainbow tables and instantly recover common passwords.
- With per-user salt + SHA-256: rainbow tables no longer help, so the attacker must brute-force each account separately, dramatically slowing progress.
- With salt and a secret pepper: the attacker needs the hidden pepper value before any brute forcing works, so the stolen hashes remain useless unless the pepper is also compromised.

## Section 4: Implementing Two-Factor Authentication (2FA)
Time-based one-time passwords (TOTP) add a "something you have" factor by generating short-lived codes from a shared secret and the current time.

In [25]:
import time
from pathlib import Path

import pyotp
import qrcode

def setup_totp(account_name: str, issuer: str = "PasswordSecurityLab") -> dict:
    """Create TOTP secret, provisioning URI, and save a QR code image."""
    secret = pyotp.random_base32()
    totp = pyotp.TOTP(secret)
    provisioning_uri = totp.provisioning_uri(name=account_name, issuer_name=issuer)

    qr = qrcode.make(provisioning_uri)
    qr_path = Path(f"{account_name.replace('@', '_at_')}_totp.png")
    qr.save(qr_path)

    return {
        "totp": totp,
        "secret": secret,
        "provisioning_uri": provisioning_uri,
        "qr_path": qr_path,
    }

def demo_totp_flow(account_name: str) -> None:
    """Show current TOTP code and verification workflow."""
    result = setup_totp(account_name)
    totp: pyotp.TOTP = result["totp"]

    current_code = totp.now()
    print("Provisioning URI:")
    print(f"  {result['provisioning_uri']}")
    print(f"QR code saved to: {result['qr_path'].resolve()}")
    print(f"Shared secret (store securely!): {result['secret']}")

    print("\nCurrent TOTP code:")
    print(f"  {current_code}")
    print(f"Valid right now: {totp.verify(current_code)}")

    time.sleep(5)
    next_code = totp.now()
    print("Next 30-second window code:")
    print(f"  {next_code}")
    print(f"Old code still valid? {totp.verify(current_code)}")

demo_totp_flow("user@example.com")

Provisioning URI:
  otpauth://totp/PasswordSecurityLab:user%40example.com?secret=O2IXA3LBEYJYH2K4LA3BDKL6MMEXF3GD&issuer=PasswordSecurityLab
QR code saved to: C:\Users\Micha\OneDrive\My Time at Goldsmiths\Year 3\Networks and System Security\E-Portfolio of Evidence\Week 03\user_at_example.com_totp.png
Shared secret (store securely!): O2IXA3LBEYJYH2K4LA3BDKL6MMEXF3GD

Current TOTP code:
  078448
Valid right now: True
Next 30-second window code:
  078448
Old code still valid? True
Next 30-second window code:
  078448
Old code still valid? True


### Why TOTP Beats SMS-Based Codes
TOTP secrets live only on your device and generate codes offline, so attackers cannot intercept them in transit. SMS codes rely on the phone network, where messages can be snooped and numbers hijacked through SIM swapping or port-out fraud, making SMS-based 2FA far easier to compromise.

## Section 5: Simulating a Brute-Force Attack

In [26]:
import itertools

COMMON_PASSWORD_LIST = [
    "password","123456","qwerty","letmein","dragon","sunshine","monkey","football","iloveyou","admin",
]

def brute_force_dictionary(target_hash: str, hash_type: str = "md5") -> None:
    """Attempt to recover the original password using a small dictionary."""
    hash_func = hash_md5 if hash_type.lower() == "md5" else hash_sha256

    for attempt in COMMON_PASSWORD_LIST:
        attempt_hash = hash_func(attempt)
        print(f"Trying '{attempt}' -> {attempt_hash}")
        if attempt_hash == target_hash:
            print(f"SUCCESS! Password is '{attempt}' using {hash_type.upper()}.")
            return

    print("No match found in dictionary.")

In [27]:
target_password = "letmein"
target_hash_md5 = hash_md5(target_password)
target_hash_sha256 = hash_sha256(target_password)

print("MD5 dictionary attack:")
brute_force_dictionary(target_hash_md5, "md5")

print("\nSHA-256 dictionary attack:")
brute_force_dictionary(target_hash_sha256, "sha256")

MD5 dictionary attack:
Trying 'password' -> 5f4dcc3b5aa765d61d8327deb882cf99
Trying '123456' -> e10adc3949ba59abbe56e057f20f883e
Trying 'qwerty' -> d8578edf8458ce06fbc5bb76a58c5ca4
Trying 'letmein' -> 0d107d09f5bbe40cade3de5c71e9e9b7
SUCCESS! Password is 'letmein' using MD5.

SHA-256 dictionary attack:
Trying 'password' -> 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Trying '123456' -> 8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3ca12020c923adc6c92
Trying 'qwerty' -> 65e84be33532fb784c48129675f9eff3a682b27168c0ea744b2cf58ee02337c5
Trying 'letmein' -> 1c8bfe8f801d79745c4631d09fff36c82aa37fc4cce4fc946683d7b336b63032
SUCCESS! Password is 'letmein' using SHA256.
